In [1]:
!pip install yfinance mysql-connector-python pandas numpy scipy matplotlib seaborn hubspot-api-client


  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)



[notice] A new release of pip is available: 24.1.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import yfinance as yf
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from hubspot import HubSpot
from hubspot.crm.contacts import SimplePublicObjectInput
import os


In [4]:
#   Connect to MySQL Database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="pass",  
    database="db_name"
)
cursor = conn.cursor()

#  Create Table (If Not Exists)
cursor.execute("""
    CREATE TABLE IF NOT EXISTS stock_prices (
        id INT AUTO_INCREMENT PRIMARY KEY,
        stock_symbol VARCHAR(10),
        date DATE,
        close_price FLOAT
    )
""")
conn.commit()
print(" MySQL Connection Successful & Table Created!")


✅ MySQL Connection Successful & Table Created!


Getting datta from Yahoo Finance API

In [ ]:
import yfinance as yf
import pandas as pd
import mysql.connector

# List of stocks to fetch (You can add more)
stock_symbols = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    port=3306,          
    user="root",
    password="pass",  
    database="db_name"
)
cursor = conn.cursor()

# Loop through each stock symbol
for stock_symbol in stock_symbols:
    print(f"Fetching data for {stock_symbol}...")

    # Fetch historical stock data (last 5 years)
    stock = yf.Ticker(stock_symbol)
    data = stock.history(period="5y")

    # Reset index to get 'Date' as a column
    data.reset_index(inplace=True)
    
    # Insert data into MySQL
    for index, row in data.iterrows():
        cursor.execute("""
            INSERT INTO stock_prices (date, stock_symbol, open_price, high_price, low_price, close_price, volume)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (row['Date'], stock_symbol, row['Open'], row['High'], row['Low'], row['Close'], row['Volume']))
    
    print(f" Inserted {len(data)} rows for {stock_symbol} into MySQL!")

# Commit changes
conn.commit()

# Close connection
cursor.close()
conn.close()

print("🎉 All stock data inserted successfully!")


We will fetch the same stocks from MySQL and process their historical returns.

In [ ]:
import pandas as pd
import mysql.connector
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

# Connect to MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="pass",
    database="db_name"
)

# Fetch stock data
query = "SELECT stock_symbol, date, close_price FROM stock_prices ORDER BY date;"
df = pd.read_sql(query, conn)

# Close MySQL connection
conn.close()

# Pivot data for easier processing
df = df.pivot(index='date', columns='stock_symbol', values='close_price')

# Calculate daily returns
returns = df.pct_change().dropna()

# Display sample data
returns.head()


We’ll generate 10,000 random portfolios and calculate expected return, risk (volatility), and Sharpe Ratio.

In [ ]:
# Number of portfolios to simulate
num_portfolios = 10000

# Store portfolio metrics
all_weights = np.zeros((num_portfolios, len(df.columns)))
ret_arr = np.zeros(num_portfolios)
vol_arr = np.zeros(num_portfolios)
sharpe_arr = np.zeros(num_portfolios)

# Get average annual returns and covariance matrix
annual_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252

# Monte Carlo Simulation
for i in range(num_portfolios):
    # Random weights for each stock
    weights = np.random.random(len(df.columns))
    weights /= np.sum(weights)  # Normalize to sum to 1
    
    # Expected return
    ret_arr[i] = np.sum(weights * annual_returns)
    
    # Expected volatility
    vol_arr[i] = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    
    # Sharpe Ratio
    sharpe_arr[i] = ret_arr[i] / vol_arr[i]
    
    # Store weights
    all_weights[i, :] = weights

# Convert results to DataFrame
portfolio_df = pd.DataFrame({
    'Return': ret_arr,
    'Volatility': vol_arr,
    'Sharpe Ratio': sharpe_arr
})

# Display top portfolios
portfolio_df.sort_values(by="Sharpe Ratio", ascending=False).head()


In [ ]:
# Find the portfolio with the highest Sharpe Ratio
max_sharpe_idx = sharpe_arr.argmax()
optimal_weights = all_weights[max_sharpe_idx]

# Display optimal weights
optimal_portfolio = pd.DataFrame({
    'Stock': df.columns,
    'Optimal Weight': optimal_weights
})

optimal_portfolio


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(vol_arr, ret_arr, c=sharpe_arr, cmap='viridis', alpha=0.7)
plt.colorbar(label='Sharpe Ratio')

# Highlight the optimal portfolio
plt.scatter(vol_arr[max_sharpe_idx], ret_arr[max_sharpe_idx], c='red', marker='*', s=200, label='Optimal Portfolio')

plt.xlabel('Volatility (Risk)')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier')
plt.legend()
plt.show()


In [ ]:
returns = df.pct_change().dropna()


In [ ]:
!pip install numpy pandas matplotlib scipy


Monte Carlo Simulation starts

In [5]:
import numpy as np
import pandas as pd

# Number of simulated portfolios
num_portfolios = 10000

# Store portfolio metrics
all_weights = np.zeros((num_portfolios, len(returns.columns)))
ret_arr = np.zeros(num_portfolios)
vol_arr = np.zeros(num_portfolios)
sharpe_arr = np.zeros(num_portfolios)

# Annualized statistics
annual_returns = returns.mean() * 252  # Annualizing daily returns
cov_matrix = returns.cov() * 252  # Annualized covariance matrix

# Monte Carlo Simulation
for i in range(num_portfolios):
    # Generate random weights for each stock
    weights = np.random.random(len(returns.columns))
    weights /= np.sum(weights)  # Normalize so they sum to 1

    # Portfolio Return
    ret_arr[i] = np.sum(weights * annual_returns)

    # Portfolio Volatility (Risk)
    vol_arr[i] = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    # Sharpe Ratio (Using risk-free rate = 3.2%)
    sharpe_arr[i] = (ret_arr[i] - 0.032) / vol_arr[i]  

    # Store weights
    all_weights[i, :] = weights

# Convert results to DataFrame
portfolio_df = pd.DataFrame({
    'Return': ret_arr,
    'Volatility': vol_arr,
    'Sharpe Ratio': sharpe_arr
})

# Display the top 5 portfolios with the highest Sharpe Ratios
portfolio_df.sort_values(by="Sharpe Ratio", ascending=False).head()


NameError: name 'returns' is not defined

Best portfolio

In [ ]:
# Find index of portfolio with highest Sharpe Ratio
max_sharpe_idx = sharpe_arr.argmax()
optimal_weights = all_weights[max_sharpe_idx]

# Display optimal weights
optimal_portfolio = pd.DataFrame({
    'Stock': returns.columns,
    'Optimal Weight': optimal_weights
})

# Display the optimal stock allocation
print(" Optimal Portfolio Allocation:")
optimal_portfolio


Visualizing Efficient Frontier

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(vol_arr, ret_arr, c=sharpe_arr, cmap='viridis', alpha=0.7)
plt.colorbar(label='Sharpe Ratio')

# Highlight the optimal portfolio
plt.scatter(vol_arr[max_sharpe_idx], ret_arr[max_sharpe_idx], c='red', marker='*', s=200, label='Optimal Portfolio')

plt.xlabel('Volatility (Risk)')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier')
plt.legend()
plt.grid(True)
plt.show()


Stock returns over time

In [ ]:
import matplotlib.pyplot as plt

# Set figure size
plt.figure(figsize=(12, 6))

# Plot daily returns for each stock
for stock in returns.columns:
    plt.plot(returns.index, returns[stock], label=stock, alpha=0.8)

# Formatting
plt.title("Stock Daily Returns Over Time")
plt.xlabel("Date")
plt.ylabel("Daily Return (%)")
plt.legend(loc='upper right')
plt.grid(True)

# Show the plot
plt.show()


Cumulatiive returns plot

In [ ]:
# Compute cumulative returns
cumulative_returns = (1 + returns).cumprod()

# Plot cumulative returns
plt.figure(figsize=(12, 6))
for stock in cumulative_returns.columns:
    plt.plot(cumulative_returns.index, cumulative_returns[stock], label=stock, alpha=0.8)

# Formatting
plt.title("Cumulative Returns Over Time")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.legend(loc='upper left')
plt.grid(True)

# Show the plot
plt.show()


Stock allocation according to best portfolio

In [ ]:
# Find the index of the best portfolio (highest Sharpe Ratio)
max_sharpe_idx = sharpe_arr.argmax()

# Extract the optimal stock weights
optimal_weights = all_weights[max_sharpe_idx]

# Create a DataFrame for visualization
optimal_portfolio = pd.DataFrame({
    'Stock': returns.columns,
    'Optimal Weight': optimal_weights
})

# Display the best portfolio allocation
print(" Optimal Portfolio Allocation:")
optimal_portfolio


Visualizing best portfolio

In [ ]:
import matplotlib.pyplot as plt

# Plot Optimal Portfolio Allocation
plt.figure(figsize=(8, 8))
plt.pie(optimal_portfolio["Optimal Weight"], labels=optimal_portfolio["Stock"], autopct='%1.1f%%', startangle=140)
plt.title("Optimal Portfolio Allocation (Best Sharpe Ratio)")
plt.show()


Setting up CRM system

In [ ]:
!pip install hubspot-api-client


In [ ]:
import hubspot
print("HubSpot API Client is installed successfully!")


In [ ]:
import requests

# Replace with your actual HubSpot OAuth Token
HUBSPOT_ACCESS_TOKEN = "your_oauth_access_token_here"

# HubSpot API URL for creating a contact
HUBSPOT_API_URL = "https://api.hubapi.com/crm/v3/objects/contacts"

# Headers for authentication
headers = {
    "Authorization": f"Bearer {HUBSPOT_ACCESS_TOKEN}",
    "Content-Type": "application/json"
}

# Contact data (test user)
contact_data = {
    "properties": {
        "email": "testuser@example.com",
        "firstname": "Test",
        "lastname": "User",
        "investment_status": "High Risk",
        "portfolio_allocation": "{'AAPL': 30%, 'TSLA': 20%, 'GOOGL': 25%, 'AMZN': 25%}"
    }
}

# Make the request
response = requests.post(HUBSPOT_API_URL, headers=headers, json=contact_data)

# Check the response
if response.status_code == 201:
    print(" Contact successfully added to HubSpot!")
else:
    print(f"❌ Error: {response.status_code}, {response.json()}")


Adding investment data in SQL

In [ ]:
import mysql.connector
import pandas as pd
import numpy as np

#  Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="pass",  
    database="db_name"
)

query = "SELECT stock_symbol, date, close_price FROM stock_prices;"
df = pd.read_sql(query, conn)

#  Compute Daily Returns
df['daily_return'] = df.groupby('stock_symbol')['close_price'].pct_change()

#  Compute Annualized Volatility (Assuming 252 trading days)
volatility = df.groupby('stock_symbol')['daily_return'].std() * np.sqrt(252)

#  Define Investment Risk Levels
def classify_risk(vol):
    if vol > 0.40:
        return "High Risk"
    elif 0.20 <= vol <= 0.40:
        return "Moderate Risk"
    else:
        return "Low Risk"

#  Apply Classification
investment_status = volatility.apply(classify_risk).reset_index()
investment_status.columns = ['stock_symbol', 'investment_status']

#  Merge with Main Data
df = df.merge(investment_status, on='stock_symbol', how='left')

#  Save Back to SQL
cursor = conn.cursor()
for index, row in df.iterrows():
    cursor.execute("""
        UPDATE stock_prices 
        SET investment_status = %s 
        WHERE stock_symbol = %s;
    """, (row['investment_status'], row['stock_symbol']))

conn.commit()
cursor.close()
conn.close()

print(" Investment Status Updated in SQL!")


In [ ]:
import mysql.connector
import pandas as pd

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="pass",  
    database="db_name"
)
cursor = conn.cursor()

# Fetch stock data
query = "SELECT stock_symbol, date, close_price FROM stock_prices"
df = pd.read_sql(query, conn)

# Calculate Daily Returns
df['daily_return'] = df.groupby('stock_symbol')['close_price'].pct_change()

# Calculate Annualized Return and Volatility
annual_returns = df.groupby('stock_symbol')['daily_return'].mean() * 252
annual_volatility = df.groupby('stock_symbol')['daily_return'].std() * (252 ** 0.5)

# Assume a risk-free rate of 3.20%
risk_free_rate = 0.032
sharpe_ratios = (annual_returns - risk_free_rate) / annual_volatility

# Classify Investment Risk
def classify_risk(volatility):
    if volatility < 0.3:
        return 'Low Risk'
    elif 0.3 <= volatility < 0.5:
        return 'Moderate Risk'
    else:
        return 'High Risk'

df['investment_status'] = df['stock_symbol'].map(lambda x: classify_risk(annual_volatility[x]))

# Update MySQL Database
for stock in df['stock_symbol'].unique():
    cursor.execute("""
        UPDATE stock_prices 
        SET annualized_return = %s, 
            annualized_volatility = %s, 
            sharpe_ratio = %s, 
            investment_status = %s 
        WHERE stock_symbol = %s
    """, (annual_returns[stock], annual_volatility[stock], sharpe_ratios[stock], classify_risk(annual_volatility[stock]), stock))

conn.commit()
cursor.close()
conn.close()

print(" Data successfully updated in MySQL!")


Fetch stock prices in python

In [ ]:
import mysql.connector
import pandas as pd

#  Connect to MySQL Database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="pass",  
    database="db_name"
)
cursor = conn.cursor()

#  Fetch stock price data
query = "SELECT date, stock_symbol, close_price FROM stock_prices ORDER BY stock_symbol, date;"
df = pd.read_sql(query, conn)

#  Close the connection
conn.close()

#  Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

#  Display DataFrame
print(df.head())


In [ ]:
#  Sort DataFrame by stock symbol and date
df = df.sort_values(by=['stock_symbol', 'date'])

#  Calculate Daily Returns
df['daily_return'] = df.groupby('stock_symbol')['close_price'].pct_change()

#  Display results
print(df.head())


In [ ]:
#  Annualized Return (Average Daily Return * 252 Trading Days)
annual_returns = df.groupby('stock_symbol')['daily_return'].mean() * 252

#  Annualized Volatility (Standard Deviation of Daily Return * sqrt(252))
annual_volatility = df.groupby('stock_symbol')['daily_return'].std() * (252 ** 0.5)

#  Store in a DataFrame
portfolio_metrics = pd.DataFrame({
    'annualized_return': annual_returns,
    'annualized_volatility': annual_volatility
})

#  Display the results
print(portfolio_metrics)


In [ ]:
#  Define Risk-Free Rate (3.2% or 0.032 in decimal)
risk_free_rate = 0.032

#  Calculate Sharpe Ratio
portfolio_metrics['sharpe_ratio'] = (portfolio_metrics['annualized_return'] - risk_free_rate) / portfolio_metrics['annualized_volatility']

#  Display results
print(portfolio_metrics)


Store to sql

In [ ]:
#  Reconnect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="pass",  
    database="db_name"
)
cursor = conn.cursor()

#  Store the updated values
for stock, row in portfolio_metrics.iterrows():
    query = """
        UPDATE stock_prices 
        SET annualized_return = %s, annualized_volatility = %s, sharpe_ratio = %s 
        WHERE stock_symbol = %s;
    """
    cursor.execute(query, (row['annualized_return'], row['annualized_volatility'], row['sharpe_ratio'], stock))

#  Commit and close connection
conn.commit()
conn.close()

print(" Sharpe Ratio and financial metrics updated in MySQL database!")


Expport Data

In [ ]:
import mysql.connector
import pandas as pd

#  Connect to MySQL Database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="pass",  
    database="db_name"
)

#  Run Query to Fetch Data
query = """
SELECT stock_symbol, 
       date, 
       close_price, 
       daily_return, 
       annualized_return, 
       annualized_volatility, 
       sharpe_ratio 
FROM stock_prices
ORDER BY stock_symbol, date;
"""
df = pd.read_sql(query, conn)

#  Save as CSV
df.to_csv("stock_data.csv", index=False)

#  Close Connection
conn.close()

print(" Dataset successfully exported as stock_data.csv!")


With investment status data

In [ ]:
import mysql.connector
import pandas as pd

#  Connect to MySQL Database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="pass",  
    database="db_name"
)

#  Fetch stock data
query = """
SELECT stock_symbol, 
       date, 
       close_price, 
       daily_return, 
       annualized_return, 
       annualized_volatility, 
       sharpe_ratio 
FROM stock_prices
ORDER BY stock_symbol, date;
"""
df = pd.read_sql(query, conn)
conn.close()

#  Function to Assign Investment Status
def classify_risk(volatility):
    if volatility < 0.20:
        return 'Low Risk'
    elif 0.20 <= volatility < 0.40:
        return 'Moderate Risk'
    else:
        return 'High Risk'

#  Apply Classification
df['investment_status'] = df['annualized_volatility'].apply(classify_risk)

#  Save Final Dataset with Investment Status
df.to_csv("stock_data_with_risk.csv", index=False)

print(" Investment Status added & dataset exported as stock_data_with_risk.csv!")


In [ ]:
df_check = pd.read_csv("stock_data_with_risk.csv")
print(df_check.head())


In [ ]:
import pandas as pd

# Sample Monte Carlo Portfolio Allocation (Replace with actual results)
optimal_allocation = {
    'AAPL': 0.25,  # 25%
    'TSLA': 0.20,  # 20%
    'GOOGL': 0.30, # 30%
    'AMZN': 0.25   # 25%
}

# Convert dictionary to DataFrame
allocation_df = pd.DataFrame(list(optimal_allocation.items()), columns=['stock_symbol', 'portfolio_allocation'])

# Convert allocation percentages to string format (for Tableau)
allocation_df['portfolio_allocation'] = (allocation_df['portfolio_allocation'] * 100).astype(str) + '%'

print(allocation_df)


In [ ]:
#  Load the dataset with Investment Status
df = pd.read_csv("stock_data_with_risk.csv")

#  Merge dataset with portfolio allocations based on stock_symbol
df = df.merge(allocation_df, on="stock_symbol", how="left")

#  Fill missing values (if any stocks were not in Monte Carlo results)
df['portfolio_allocation'] = df['portfolio_allocation'].fillna("0%")  

#  Save the final dataset
df.to_csv("stock_data_with_risk_and_allocations.csv", index=False)

print(" Portfolio allocation added! Final dataset exported as stock_data_with_risk_and_allocations.csv")


In [ ]:

import pandas as pd

#  Step 1: Store optimal allocation into a DataFrame
optimal_allocation = {
    'stock_symbol': ['AAPL', 'AMZN', 'GOOGL', 'MSFT', 'TSLA'],
    'portfolio_allocation': [0.247438, 0.009744, 0.195182, 0.108124, 0.439513]
}

allocation_df = pd.DataFrame(optimal_allocation)

#  Convert allocation values to percentage format for readability
allocation_df['portfolio_allocation'] = (allocation_df['portfolio_allocation'] * 100).round(2).astype(str) + '%'

#  Display the DataFrame
import ace_tools as tools
tools.display_dataframe_to_user(name="Optimal Portfolio Allocation", dataframe=allocation_df)


In [ ]:
#  Step 2: Load the existing stock dataset
df = pd.read_csv("stock_data_with_risk.csv")  # Load the latest dataset

#  Merge dataset with portfolio allocations based on stock_symbol
df = df.merge(allocation_df, on="stock_symbol", how="left")

#  Fill missing values (for stocks not in portfolio allocation)
df['portfolio_allocation'] = df['portfolio_allocation'].fillna("0%")

#  Save the final dataset
df.to_csv("stock_data_with_risk_and_allocations.csv", index=False)

print(" Portfolio allocation added! Final dataset exported as stock_data_with_risk_and_allocations.csv")
